```markdown
## Configuration Interaction (CI) for H2 Molecule in STO-3G Basis Set

In this Jupyter Notebook, we perform a Configuration Interaction (CI) calculation for the H2 molecule using the STO-3G basis set. The Hamiltonian is provided in the .LVCORR format, compatible with DIRAC22.

### Steps Involved:

1. **Reading the Hamiltonian File:**
    - The Hamiltonian file `FCIDUMP_lvcorr` is read and processed to extract the integrals.

2. **Defining Slater-Condon Rules:**
    - Functions `SC_0`, `SC_1`, and `SC_2` are defined to calculate matrix elements based on Slater-Condon rules for different excitation levels.

3. **Matrix Elements Calculation:**
    - The `Matrix_elements` function calculates the matrix elements for the given states using the integrals.

4. **Eigenvalue and Eigenvector Calculation:**
    - The matrix elements are used to construct a Hamiltonian matrix, and its eigenvalues and eigenvectors are computed.

### Variables:

- **eigenvalues**: `numpy.ndarray`
  - Array of eigenvalues obtained from the Hamiltonian matrix.
  - Value: `array([-1.85728905, -1.24464989, -0.88279394, -0.22503878])`

- **eigenvectors**: `numpy.ndarray`
  - Array of eigenvectors corresponding to the eigenvalues.
  - Value: 
     ```python
     array([[-9.93759745e-01,  3.04776736e-14,  1.04513242e-14, -1.11541781e-01],
              [-2.66037192e-14, -7.07106781e-01, -7.07106781e-01, -2.24265051e-14],
              [ 1.11541781e-01, -2.74225087e-14,  5.52891066e-14, -9.93759745e-01],
              [-2.06007594e-14, -7.07106781e-01,  7.07106781e-01,  5.65836216e-14]])
     ```

- **integral_dict**: `dict`
  - Dictionary containing the integrals extracted from the Hamiltonian file.
  - Example Value: `{'(1, 1, 1, 1)': 0.6757089065142776, '(2, 2, 2, 2)': 0.6757089065142776, ...}`

- **matrix_elements**: `numpy.ndarray`
  - Matrix of Hamiltonian elements calculated for the given states.
  - Value: 
     ```python
     array([[-1.83698129e+00, -6.85456235e-14,  1.80927976e-01, -6.85456235e-14],
              [-1.63030983e-14, -1.06372191e+00, -4.18315296e-14, -1.80927976e-01],
              [ 1.80927976e-01,  1.05343463e-14, -2.45346532e-01,  1.05343463e-14],
              [-1.63030983e-14, -1.80927976e-01, -4.18315296e-14, -1.06372191e+00]])
     ```

### Conclusion:

The CI calculation for the H2 molecule in the STO-3G basis set has been successfully performed. The eigenvalues and eigenvectors of the Hamiltonian matrix provide insights into the electronic structure of the molecule.
```

In [26]:
file_path = "C:/Users/prita/Desktop/DIRAC_H2_FCI/QAE_finite_field/FCIDUMP_lvcorr"
# /content/drive/MyDrive/Colab Notebooks/QAE_finite_field/FCIDUMP_dossss
# Read the file and store lines
with open(file_path, "r") as file:
    lines = file.readlines()

# Ignore the first 6 lines
lines = lines[6:]

# Process the remaining lines
integral_dict = {}
for line in lines:
    parts = line.split()
    if len(parts) == 6:  # Ensure there are exactly 5 columns
        integral_value = float(parts[0])
        p, q, r, s = map(int, parts[2:])
        key = f"({p}, {q}, {r}, {s})"
        integral_dict[key] = integral_value

# Print the parsed data
for key, value in integral_dict.items():
    print(f"{key}: {value}")

(1, 1, 1, 1): 0.6757089065142776
(2, 2, 2, 2): 0.6757089065142776
(1, 1, 3, 1): -1.298977304815158e-14
(2, 2, 4, 2): -1.298977304815158e-14
(1, 1, 1, 3): -1.300252004717957e-14
(2, 2, 2, 4): -1.300252004717957e-14
(1, 1, 3, 3): 0.6645840965175442
(2, 2, 4, 4): 0.6645840965175442
(1, 1, 2, 2): 0.6757089065142776
(2, 2, 1, 1): 0.6757089065142776
(1, 1, 4, 2): -1.298977304815158e-14
(2, 2, 3, 1): -1.298977304815158e-14
(1, 1, 2, 4): -1.300252004717957e-14
(2, 2, 1, 3): -1.300252004717957e-14
(1, 1, 4, 4): 0.6645840965175442
(2, 2, 3, 3): 0.6645840965175442
(3, 1, 1, 1): -1.3312176257486964e-14
(4, 2, 2, 2): -1.3312176257486964e-14
(3, 1, 3, 1): 0.18092797565020144
(4, 2, 4, 2): 0.18092797565020144
(3, 1, 1, 3): 0.18092797565020197
(4, 2, 2, 4): 0.18092797565020197
(3, 1, 3, 3): 1.3525268382765016e-14
(4, 2, 4, 4): 1.3525268382765016e-14
(3, 1, 2, 2): -1.3312176257486964e-14
(4, 2, 1, 1): -1.3312176257486964e-14
(3, 1, 4, 2): 0.180927975650202
(4, 2, 3, 1): 0.180927975650202
(3, 1, 2, 4): 

In [27]:
def SC_0(ref_state, integral_dict):
    sum_one = 0
    for i in ref_state:
        sum_one += integral_dict[f"({i}, {i}, {0}, {0})"]
    sum_two = 0
    for i in ref_state:
        for j in ref_state:
            if i != j:
                try:
                    sum_two += (integral_dict[f"({i}, {i}, {j}, {j})"])
                    # print(f"({i}, {i}, {j}, {j})")
                except KeyError:
                    pass  # Ignore the error and continue
                
                try:
                    sum_two -= (integral_dict[f"({i}, {j}, {j}, {i})"])
                    # print(f"({i}, {j}, {j}, {i})")
                except KeyError:
                    pass  # Ignore the error and continue
    # return sum_one+(0.5*sum_two)+integral_dict["(0, 0, 0, 0)"]
    return sum_one+(0.5*sum_two)

In [28]:
def SC_1(state_bra, state_ket, integral_dict):
    sum_one  = 0
    sum_two = 0
    p = list(set(state_ket) - set(state_bra))
    m = list(set(state_bra) - set(state_ket))
    for i in state_bra:
        for j in state_ket:
            if i not in state_ket and j not in state_bra:
                sum_one += integral_dict[f"({i}, {j}, {0}, {0})"]
    for i in state_bra:
        try:
            sum_two += integral_dict[f"({m[0]}, {p[0]}, {i}, {i})"]
        except KeyError:
            pass
        
        try:
            sum_two += integral_dict[f"({m[0]}, {i}, {i}, {p[0]})"]
        except KeyError:
            pass
    # return sum_one+sum_two+integral_dict["(0, 0, 0, 0)"]
    return sum_one+sum_two


In [29]:
def SC_2(state_bra, state_ket, integral_dict):
    p = list(set(state_ket) - set(state_bra))
    m = list(set(state_bra) - set(state_ket))
    sum_two = 0
    try: 
        sum_two += integral_dict[f"({m[0]}, {p[0]}, {m[1]}, {p[1]})"]
    except KeyError:
        pass
    try:
        sum_two += -integral_dict[f"({m[0]}, {p[1]}, {m[1]}, {p[0]})"]
    except KeyError:
        pass
    # return sum_two + integral_dict["(0, 0, 0, 0)"]
    return sum_two 


In [30]:
def Matrix_elements(state_bra, state_ket, integral_dict):
    # Find  difference elements
    difference = list(set(state_ket) - set(state_bra))
    if len(difference) == 0:
        value = SC_0(state_bra, integral_dict)# slater condon rule for 0 difference
        return value
    elif len(difference) == 1:
        value = SC_1(state_bra, state_ket, integral_dict)# slater condon rule for 1 difference
        return value
    elif len(difference) == 2:
        value = SC_2(state_bra, state_ket, integral_dict)# slater condon rule for 2 difference
        return value
    else:
        return 0

In [31]:
import numpy as np

state_1 = [1, 2]
state_2 = [3, 2]
state_3 = [3, 4] 
state_4 = [1, 4]
states = [state_1, state_2, state_3, state_4]
# Create a null matrix of dimension equal to the number of states
num_states = len(states)
matrix_elements = np.zeros((num_states, num_states))

# Calculate the matrix elements
for i, state_bra in enumerate(states):
    for j, state_ket in enumerate(states):
        matrix_elements[i, j] = Matrix_elements(state_bra, state_ket, integral_dict)

print(matrix_elements)
    


[[-1.83698129e+00 -6.85456235e-14  1.80927976e-01 -6.85456235e-14]
 [-1.63030983e-14 -1.06372191e+00 -4.18315296e-14 -1.80927976e-01]
 [ 1.80927976e-01  1.05343463e-14 -2.45346532e-01  1.05343463e-14]
 [-1.63030983e-14 -1.80927976e-01 -4.18315296e-14 -1.06372191e+00]]


In [32]:
import numpy as np

#calculate the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(matrix_elements)
print("Eigenvalues:")
print(eigenvalues)

Eigenvalues:
[-1.85728905 -1.24464989 -0.88279394 -0.22503878]


In [33]:
eigenvalues + integral_dict["(0, 0, 0, 0)"]

array([-1.13732005, -0.52468089, -0.16282494,  0.49493022])